In [15]:
# Output directory
import spacy
from pathlib import Path
from spacy.training import Example
from spacy import displacy
from IPython.core.display import display, HTML
from IPython.display import clear_output
import requests

In [2]:
output_dir=Path('/Users/omerfaruk.davarci/Documents/MetabolysmAnalysisAPI/content2')

# Loading the model from the directory
print("Loading from", output_dir)
nlp = spacy.load(output_dir)

Loading from /Users/omerfaruk.davarci/Documents/MetabolysmAnalysisAPI/content2


In [3]:
nlp_disease = spacy.load("en_ner_bc5cdr_md")

In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'gproject.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import re
from IPython.display import clear_output

from articles.models import Article, Pathway, PathwayDiseaseRelation
from django.db.models import Exists

In [5]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download([
    "names",
    "stopwords",
    "state_union",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",])
sia = SentimentIntensityAnalyzer()
from textblob import TextBlob

[nltk_data] Downloading package names to
[nltk_data]     /Users/omerfaruk.davarci/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/omerfaruk.davarci/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     /Users/omerfaruk.davarci/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/omerfaruk.davarci/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/omerfaruk.davarci/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/omerfaruk.davarci/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
#s = flair.data.Sentence(sentence)
#flair_sentiment.predict(s)
#total_sentiment = s.labels
#total_sentiment

2021-06-24 07:04:49,767 loading file /Users/omerfaruk.davarci/.flair/models/sentiment-en-mix-distillbert_4.pt


In [2]:
has_pathway = Article.objects.exclude(pathways=None).all()
has_pathway = has_pathway.filter(pathways__name__exact='Urea cycle').all()
[entry for entry in has_pathway]

[<Article: Article object (465146)>,
 <Article: Article object (465145)>,
 <Article: Article object (465144)>,
 <Article: Article object (465143)>,
 <Article: Article object (465142)>,
 <Article: Article object (465141)>,
 <Article: Article object (465140)>,
 <Article: Article object (465139)>,
 <Article: Article object (465138)>,
 <Article: Article object (465137)>,
 <Article: Article object (465136)>,
 <Article: Article object (465135)>,
 <Article: Article object (465134)>,
 <Article: Article object (465133)>,
 <Article: Article object (465132)>,
 <Article: Article object (465131)>,
 <Article: Article object (465130)>,
 <Article: Article object (465129)>,
 <Article: Article object (465128)>,
 <Article: Article object (465127)>,
 <Article: Article object (465126)>,
 <Article: Article object (465125)>,
 <Article: Article object (465124)>,
 <Article: Article object (465123)>,
 <Article: Article object (465122)>,
 <Article: Article object (465121)>,
 <Article: Article object (465120)>,
 

In [3]:
d = PathwayDiseaseRelation.objects.all()
[entry for entry in d]

[<PathwayDiseaseRelation: PathwayDiseaseRelation object (1085)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (369)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (370)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (547)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (548)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2056)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2057)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2058)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2059)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2069)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2070)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2071)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2072)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2073)>,
 <PathwayDiseaseRelation: PathwayDiseaseRelation object (2074)>,
 <PathwayDiseaseRelation: Pat

In [9]:
z = 0
for d in has_pathway:
    print(d.id)
    doc = nlp(d.abstract_text)
    doc_disease = nlp_disease(d.abstract_text)
        
    if len(doc.ents) > 0:
        for entity in doc.ents:
            sia_score = sia.polarity_scores(entity.sent.text)
            textblob_score = TextBlob(entity.sent.text).sentiment
            
            s = flair.data.Sentence(entity.sent.text)
            flair_sentiment.predict(s)
            flair_score = s.labels
            print("\n\n#######################")
            print('Entity extracted : ', entity.text)
            print("\n\n#######################")
            print('Sentence extracted from : ', entity.sent)
            print("\n\n#######################")
            print(sia_score)
            print(textblob_score)
            print(flair_score)
            
            if len(doc_disease.ents) > 0:
                for disease in doc_disease.ents:
                    if disease.label_ == 'DISEASE':
                        print(sia_score, flair_score, textblob_score)
                        obj, created = PathwayDiseaseRelation.objects.get_or_create(pathway_name=entity.text.lower(), disease_name=disease.text.lower(), article_id=d.id, sentence=entity.sent.text)
                        if created:
                            obj.sia_score = sia_score['compound']
                            obj.textblob_polarity = textblob_score.polarity
                            obj.textblob_subjectivity = textblob_score.subjectivity
                            obj.flair_score = flair_score[0].score
                            obj.save()
    z += 1


465146


#######################
Entity extracted :  Urea cycle


#######################
Sentence extracted from :  <h4>Background</h4>Urea cycle disorders (UCD) and organic acid disorders classically present in the neonatal period.


#######################
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentiment(polarity=0.0, subjectivity=0.0)
[POSITIVE (0.9304)]
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [POSITIVE (0.9304)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [POSITIVE (0.9304)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [POSITIVE (0.9304)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [POSITIVE (0.9304)] Sentiment(polarity=0.0, subjectivity=0.0)
465145


#######################
Entity extracted :  urea cycle


#######################
Sentence extracted from :  <h4>Background</h4>The amino acid l-citrull

{'neg': 0.112, 'neu': 0.888, 'pos': 0.0, 'compound': -0.34} [NEGATIVE (0.9998)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.112, 'neu': 0.888, 'pos': 0.0, 'compound': -0.34} [NEGATIVE (0.9998)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.112, 'neu': 0.888, 'pos': 0.0, 'compound': -0.34} [NEGATIVE (0.9998)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.112, 'neu': 0.888, 'pos': 0.0, 'compound': -0.34} [NEGATIVE (0.9998)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.112, 'neu': 0.888, 'pos': 0.0, 'compound': -0.34} [NEGATIVE (0.9998)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.112, 'neu': 0.888, 'pos': 0.0, 'compound': -0.34} [NEGATIVE (0.9998)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.112, 'neu': 0.888, 'pos': 0.0, 'compound': -0.34} [NEGATIVE (0.9998)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.112, 'neu': 0.888, 'pos': 0.0, 'compound': -0.34} [NEGATIVE (0.9998)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.112, 'neu': 0.

{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.4215} [NEGATIVE (0.6575)] Sentiment(polarity=-0.016666666666666677, subjectivity=0.16666666666666666)
{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.4215} [NEGATIVE (0.6575)] Sentiment(polarity=-0.016666666666666677, subjectivity=0.16666666666666666)
{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.4215} [NEGATIVE (0.6575)] Sentiment(polarity=-0.016666666666666677, subjectivity=0.16666666666666666)
{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.4215} [NEGATIVE (0.6575)] Sentiment(polarity=-0.016666666666666677, subjectivity=0.16666666666666666)
{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.4215} [NEGATIVE (0.6575)] Sentiment(polarity=-0.016666666666666677, subjectivity=0.16666666666666666)
{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.4215} [NEGATIVE (0.6575)] Sentiment(polarity=-0.016666666666666677, subjectivity=0.16666666666666666)
{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound':

{'neg': 0.387, 'neu': 0.613, 'pos': 0.0, 'compound': -0.743} [NEGATIVE (0.9962)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.387, 'neu': 0.613, 'pos': 0.0, 'compound': -0.743} [NEGATIVE (0.9962)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.387, 'neu': 0.613, 'pos': 0.0, 'compound': -0.743} [NEGATIVE (0.9962)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.387, 'neu': 0.613, 'pos': 0.0, 'compound': -0.743} [NEGATIVE (0.9962)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.387, 'neu': 0.613, 'pos': 0.0, 'compound': -0.743} [NEGATIVE (0.9962)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.387, 'neu': 0.613, 'pos': 0.0, 'compound': -0.743} [NEGATIVE (0.9962)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.387, 'neu': 0.613, 'pos': 0.0, 'compound': -0.743} [NEGATIVE (0.9962)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.387, 'neu': 0.613, 'pos': 0.0, 'compound': -0.743} [NEGATIVE (0.9962)] Sentiment(polarity=0.0, subjectivity=0.0)


##############

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9271)] Sentiment(polarity=-0.16666666666666666, subjectivity=0.7333333333333334)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9271)] Sentiment(polarity=-0.16666666666666666, subjectivity=0.7333333333333334)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9271)] Sentiment(polarity=-0.16666666666666666, subjectivity=0.7333333333333334)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9271)] Sentiment(polarity=-0.16666666666666666, subjectivity=0.7333333333333334)


#######################
Entity extracted :  polyamine synthesis


#######################
Sentence extracted from :  The key enzymes used in the urea cycle, namely arginase, ornithine transcarbamylase, argininosuccinate synthase and argininosuccinate lyase, in addition to two rate limiting enzymes required for polyamine synthesis (ornithine decarboxylase and s-adenosylmethionine decarboxylase) are poorl

...

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9971)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9971)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9971)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9971)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9971)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9971)] Sentiment(polarity=0.0, subjectivity=0.0)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} [NEGATIVE (0.9971)] Sentiment(polarity=0.0, subjectivity=0.0)
465059


#######################
Entity extracted :  urea cycle


#######################
Sentence extracted from :  The ornithine-urea cycle (OUC) belongs to the cornerstone of the metabo

{'neg': 0.0, 'neu': 0.721, 'pos': 0.279, 'compound': 0.8519} [POSITIVE (0.9999)] Sentiment(polarity=0.1, subjectivity=0.3)
{'neg': 0.0, 'neu': 0.721, 'pos': 0.279, 'compound': 0.8519} [POSITIVE (0.9999)] Sentiment(polarity=0.1, subjectivity=0.3)
{'neg': 0.0, 'neu': 0.721, 'pos': 0.279, 'compound': 0.8519} [POSITIVE (0.9999)] Sentiment(polarity=0.1, subjectivity=0.3)
{'neg': 0.0, 'neu': 0.721, 'pos': 0.279, 'compound': 0.8519} [POSITIVE (0.9999)] Sentiment(polarity=0.1, subjectivity=0.3)
{'neg': 0.0, 'neu': 0.721, 'pos': 0.279, 'compound': 0.8519} [POSITIVE (0.9999)] Sentiment(polarity=0.1, subjectivity=0.3)
{'neg': 0.0, 'neu': 0.721, 'pos': 0.279, 'compound': 0.8519} [POSITIVE (0.9999)] Sentiment(polarity=0.1, subjectivity=0.3)
{'neg': 0.0, 'neu': 0.721, 'pos': 0.279, 'compound': 0.8519} [POSITIVE (0.9999)] Sentiment(polarity=0.1, subjectivity=0.3)
465057


#######################
Entity extracted :  urea cycle


#######################
Sentence extracted from :  Here, we demonstrate t

{'neg': 0.17, 'neu': 0.83, 'pos': 0.0, 'compound': -0.3947} [NEGATIVE (0.9996)] Sentiment(polarity=0.1, subjectivity=0.5)
{'neg': 0.17, 'neu': 0.83, 'pos': 0.0, 'compound': -0.3947} [NEGATIVE (0.9996)] Sentiment(polarity=0.1, subjectivity=0.5)
{'neg': 0.17, 'neu': 0.83, 'pos': 0.0, 'compound': -0.3947} [NEGATIVE (0.9996)] Sentiment(polarity=0.1, subjectivity=0.5)
{'neg': 0.17, 'neu': 0.83, 'pos': 0.0, 'compound': -0.3947} [NEGATIVE (0.9996)] Sentiment(polarity=0.1, subjectivity=0.5)


#######################
Entity extracted :  urea cycle


#######################
Sentence extracted from :  As therapies for hyperammonemia associated with urea cycle dysfunction have improved, chronic complications, such as liver disease, have become increasingly apparent in individuals with UCDs.


#######################
{'neg': 0.097, 'neu': 0.796, 'pos': 0.107, 'compound': 0.0772}
Sentiment(polarity=0.025, subjectivity=0.425)
[NEGATIVE (0.8018)]
{'neg': 0.097, 'neu': 0.796, 'pos': 0.107, 'compound': 



#######################
Entity extracted :  urea cycle


#######################
Sentence extracted from :  Since the toxicity of ammonium acetate is lower in rats with galactosamine hepatitis, in which the activities of the liver urea cycle enzymes are reduced, it seems likely that ammonia detoxication in galactosamine-poisoned rat liver partly bypasses the urea cycle.


#######################
{'neg': 0.053, 'neu': 0.947, 'pos': 0.0, 'compound': -0.296}
Sentiment(polarity=0.0, subjectivity=1.0)
[NEGATIVE (0.9975)]
{'neg': 0.053, 'neu': 0.947, 'pos': 0.0, 'compound': -0.296} [NEGATIVE (0.9975)] Sentiment(polarity=0.0, subjectivity=1.0)


#######################
Entity extracted :  urea cycle


#######################
Sentence extracted from :  Since the toxicity of ammonium acetate is lower in rats with galactosamine hepatitis, in which the activities of the liver urea cycle enzymes are reduced, it seems likely that ammonia detoxication in galactosamine-poisoned rat liver partly byp

{'neg': 0.089, 'neu': 0.911, 'pos': 0.0, 'compound': -0.5719} [POSITIVE (0.9924)] Sentiment(polarity=0.05333333333333334, subjectivity=0.37999999999999995)
54252


#######################
Entity extracted :  urea cycle


#######################
Sentence extracted from :  These alterations do not therefore appear to be related to the limitation of ammonia detoxication through the urea cycle.


#######################
{'neg': 0.109, 'neu': 0.891, 'pos': 0.0, 'compound': -0.296}
Sentiment(polarity=0.0, subjectivity=0.4)
[NEGATIVE (0.9981)]
{'neg': 0.109, 'neu': 0.891, 'pos': 0.0, 'compound': -0.296} [NEGATIVE (0.9981)] Sentiment(polarity=0.0, subjectivity=0.4)
18447


#######################
Entity extracted :  urea cycle


#######################
Sentence extracted from :  The urea cycle enzymes, carbamoyl-P-synthetase, ornithine transcarbamylase, arginase and other enzymes related to ammonia metabolism, such as glutamate dehydrogenase, glutamine synthetase and alanine and aspartate amin